In [1]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

/home/ubuntu/Qwen-Image-Edit-Angles


In [2]:
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import ExperimentRegistry


# experiment_names = ExperimentRegistry.keys()
experiment_names = [
    "qwen_base",
    # "qwen_fa3",
    # "qwen_aot",
    # "qwen_fa3_aot",
    # "qwen_fa3_aot_int8",
    # "qwen_fa3_aot_fp8",
    # "qwen_fuse",
    "qwen_base_3step",
    "qwen_base_2step",
    "qwen_lightning_lora",
    "qwen_lightning_lora_3step",
    "qwen_lightning_lora_2step",
]

report_dir = ExperimentConfig().report_dir

experiment_outputs = {}
num_outputs = {}
for name in experiment_names:
    output_dir = report_dir / f"{name}_outputs"
    all_output_paths = sorted(list(output_dir.glob("*.jpg")))
    experiment_outputs[name] = all_output_paths
    num_outputs[name] = len(all_output_paths)




/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
TMA benchmarks will be running without grid constant TMA descriptor.
2025-11-13 17:17:45.758469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the envi

In [23]:
experiment_outputs["qwen_base"][0]

PosixPath('reports/qwen_base_outputs/000.jpg')

In [ ]:

import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms

# Initialize LPIPS model
loss_fn = lpips.LPIPS(net='alex')  # or 'vgg' or 'squeeze'
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# Transform to convert PIL images to tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def calculate_lpips_scores(base_paths, compare_paths):
    """Calculate LPIPS scores between two sets of images."""
    scores = []
    
    # Get the minimum number of images available
    num_images = min(len(base_paths), len(compare_paths))
    
    for idx in range(num_images):
        # Load images
        img1 = Image.open(base_paths[idx]).convert('RGB')
        img2 = Image.open(compare_paths[idx]).convert('RGB')
        
        # Resize if dimensions don't match
        if img1.size != img2.size:
            img2 = img2.resize(img1.size, Image.LANCZOS)
        
        # Transform to tensors
        img1_tensor = transform(img1).unsqueeze(0)
        img2_tensor = transform(img2).unsqueeze(0)
        
        if torch.cuda.is_available():
            img1_tensor = img1_tensor.cuda()
            img2_tensor = img2_tensor.cuda()
        
        # Calculate LPIPS
        with torch.no_grad():
            score = loss_fn(img1_tensor, img2_tensor)
        
        scores.append(score.item())
    
    return scores

# Define experiment sets
experiment_sets = {
    'qwen_base': {
        '4step': 'qwen_base',
        '3step': 'qwen_base_3step',
        '2step': 'qwen_base_2step'
    },
    'qwen_lightning_lora': {
        '4step': 'qwen_lightning_lora',
        '3step': 'qwen_lightning_lora_3step',
        '2step': 'qwen_lightning_lora_2step'
    }
}

# Calculate LPIPS scores for each set
results = {}

for set_name, experiments in experiment_sets.items():
    print(f"\nProcessing {set_name}...")
    
    # Get image paths
    base_4step_paths = experiment_outputs[experiments['4step']]
    step_3_paths = experiment_outputs[experiments['3step']]
    step_2_paths = experiment_outputs[experiments['2step']]
    
    # Calculate LPIPS scores
    print(f"Calculating LPIPS: 4-step vs 3-step...")
    scores_4vs3 = calculate_lpips_scores(base_4step_paths, step_3_paths)
    
    print(f"Calculating LPIPS: 4-step vs 2-step...")
    scores_4vs2 = calculate_lpips_scores(base_4step_paths, step_2_paths)
    
    # Create results dataframe
    results_df = pd.DataFrame({
        'comparison': ['4step_vs_3step', '4step_vs_2step'],
        'mean_lpips': [
            np.mean(scores_4vs3),
            np.mean(scores_4vs2)
        ],
        'std_lpips': [
            np.std(scores_4vs3),
            np.std(scores_4vs2)
        ],
        'num_samples': [
            len(scores_4vs3),
            len(scores_4vs2)
        ]
    })
    
    # Save to CSV
    csv_path = report_dir / f"lpips_scores_{set_name}.csv"
    results_df.to_csv(csv_path, index=False)
    
    print(f"\nResults for {set_name}:")
    print(results_df)
    print(f"\nSaved to: {csv_path}")
    
    results[set_name] = results_df

print("\n" + "="*60)
print("LPIPS Analysis Complete!")
print("="*60)



Processing rFID for qwen_base...
Calculating rFID: 4-step vs 3-step...


In [4]:
import math
from PIL import Image
import numpy as np
from pathlib import Path

comparison_dir = report_dir / "comparison_grid"
comparison_dir.mkdir(exist_ok=True, parents=True)


comparable_outputs = min(num_outputs.values())
num_experiments = len(experiment_names)

# For each output index, create a comparison grid
for idx in range(comparable_outputs):
    cols = int(math.ceil(math.sqrt(num_experiments)))
    rows = int(math.ceil(num_experiments / cols ))
    
    fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 6 * rows))
    
    # Flatten axes array for easier indexing
    if num_experiments == 1:
        axes = np.array([axes])
    else:
        axes = axes.flatten() if isinstance(axes, np.ndarray) else np.array([axes])
    
    # Plot each experiment's output for this index
    for exp_idx, exp_name in enumerate(experiment_names):
        ax = axes[exp_idx]
        
        # Check if this experiment has an output at this index
        if idx < num_outputs[exp_name]:
            img_path = experiment_outputs[exp_name][idx]
            img = Image.open(img_path)
            ax.imshow(img)
            ax.set_title(exp_name, fontsize=14, fontweight='bold')
        else:
            # No output for this experiment at this index
            ax.text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
            ax.set_title(exp_name, fontsize=14, fontweight='bold', color='gray')
        
        ax.axis('off')
    
    # Hide any unused subplots
    for exp_idx in range(num_experiments, len(axes)):
        axes[exp_idx].axis('off')
    
    # Add a main title for the figure
    fig.suptitle(f'Output Comparison - Index {idx:03d}', fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    
    # Save the figure
    output_path = comparison_dir / f"comparison_{idx:03d}.jpg"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved comparison grid {idx + 1}/{comparable_outputs} to {output_path}")

print(f"\nAll comparison grids saved to {comparison_dir}")


Saved comparison grid 1/32 to reports/comparison_grid/comparison_000.jpg
Saved comparison grid 2/32 to reports/comparison_grid/comparison_001.jpg
Saved comparison grid 3/32 to reports/comparison_grid/comparison_002.jpg
Saved comparison grid 4/32 to reports/comparison_grid/comparison_003.jpg
Saved comparison grid 5/32 to reports/comparison_grid/comparison_004.jpg
Saved comparison grid 6/32 to reports/comparison_grid/comparison_005.jpg
Saved comparison grid 7/32 to reports/comparison_grid/comparison_006.jpg
Saved comparison grid 8/32 to reports/comparison_grid/comparison_007.jpg
Saved comparison grid 9/32 to reports/comparison_grid/comparison_008.jpg
Saved comparison grid 10/32 to reports/comparison_grid/comparison_009.jpg
Saved comparison grid 11/32 to reports/comparison_grid/comparison_010.jpg
Saved comparison grid 12/32 to reports/comparison_grid/comparison_011.jpg
Saved comparison grid 13/32 to reports/comparison_grid/comparison_012.jpg
Saved comparison grid 14/32 to reports/comparis